In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import timedelta
from datetime import datetime
import snowFun
import dask.array
# %matplotlib widget

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "01_rgi60_Alaska", "01_rgi60_Alaska.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
print(rgis_to_analyze_df[:10])

3031
               RGIId     Area
829   RGI60-01.20889    [2.0]
254   RGI60-01.11632    [2.0]
153   RGI60-01.08203  [2.002]
2486  RGI60-01.25183  [2.002]
2851  RGI60-01.11595  [2.003]
2207  RGI60-01.09047  [2.004]
557   RGI60-01.03614  [2.004]
40    RGI60-01.05487  [2.004]
417   RGI60-01.03088  [2.005]
1695  RGI60-01.10867  [2.005]


In [27]:

# subset rgi to single outline, by choosing rgiid or rgi name
rgiid = "RGI60-01.17423"

# quickly grab glacier area
ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

# choose how much to coarsen (more coarse for bigger glaciers) 
if ga>1000:
    scale=5
else:
    scale=3

# set folder
if validation_only:
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
else:
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# get geometry
rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
single_geometry = rgi_single.geometry

# print progress
print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")

# single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier

# open glacier mask
glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier

# open dem
xr_dem = snowFun.get_year_DEM(single_geometry, 2018, smoothed=0)

# open data
path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{2018}_daily_AAs_coarse.nc")
snow = xr.open_dataset(path_open, chunks={'time':10})




Starting 3031 of 3031: RGI60-01.17423  512.357 km2


In [28]:
print(glacier_mask.shape)
print(xr_dem.shape)
print(snow.dims)

(3704, 3767)
(1, 3704, 3767)
Frozen({'time': 122, 'y': 1234, 'x': 1255})


In [29]:
mask = glacier_mask.sel({"x":snow.x, "y":snow.y})
dem = xr_dem.sel({"x":snow.x, "y":snow.y})
print(mask.shape)
print(dem.shape)

(1234, 1255)
(1, 1234, 1255)


In [25]:
print(test.x.values)
print(snow.x.values)
print(snow.head())
print(test.head())

[647305. 647335. 647365. ... 684865. 684895. 684925.]
[647305. 647335. 647365. ... 684865. 684895. 684925.]
<xarray.Dataset>
Dimensions:      (time: 5, y: 5, x: 5)
Coordinates:
  * x            (x) float64 6.473e+05 6.473e+05 6.474e+05 6.474e+05 6.474e+05
  * y            (y) float64 1.331e+06 1.331e+06 1.331e+06 1.331e+06 1.331e+06
    spatial_ref  int32 ...
  * time         (time) datetime64[ns] 2018-05-01 2018-05-03 ... 2018-05-08
Data variables:
    class        (time, y, x) uint8 dask.array<chunksize=(5, 5, 5), meta=np.ndarray>
<xarray.DataArray 'glacier' (y: 5, x: 5)>
dask.array<getitem, shape=(5, 5), dtype=bool, chunksize=(5, 5), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 6.473e+05 6.473e+05 6.474e+05 6.474e+05 6.474e+05
  * y            (y) float64 1.331e+06 1.331e+06 1.331e+06 1.331e+06 1.331e+06
    spatial_ref  int32 ...
Attributes:
    res:          [10 10]
    crs:          EPSG:3338
    transform:    [ 10   0   0   0 -10   0]
    long_name:    RGI6